In [ ]:
!pip install boto3 --upgrade
!pip install sagemaker --upgrade

In [1]:
import boto3
import json
import sagemaker
from IPython.display import HTML

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()
sm_client = boto3.Session().client('sagemaker')

Setup Streamlit environemnt. Below assumes you have extracted code to /home/sagemaker-user/bedrock_onboarding directory

```bash
cd /home/sagemaker-user/aws-notebooklm
conda env create -f environment.yml
source activate aws-notebooklm-app
```

If you are running this in new SageMaker Stuido, perform following actions

```bash

Create a folder .streamlit under applications folder
mkdir .streamlit

In this folder create a file called config.toml and copy the following contents to that folder
touch config.toml

[server]
enableXsrfProtection = false
enableCORS = false

[browser]
gatherUsageStats = false
```
### Setup Environment variables
Setup the environment variables in .env file under application folder. You can setup Region, Authentication mechanism and other variables here.

## Run application 
Run the following command to launch demo application. This runs the application to run on port 8081, restrict maximum file size for upload to 10MB. Both settings can be changed.

```bash
streamlit run AWS_Podcast_Generator.py --server.port 6006 --server.maxUploadSize 10
```

In [3]:
#Read Studio meta data to get Domain, User Profile details
with open('/opt/ml/metadata/resource-metadata.json','r') as cf:
    meta_data = json.load(cf)

studio_domain = meta_data['DomainId']
application_type = meta_data['AppType']
studio_region = sagemaker_session.boto_region_name
role_arn = sagemaker_session.get_caller_identity_arn()
studio_type = 'Studio' if application_type == 'JupyterLab' else 'Studio Classic'
domian_settings = sm_client.describe_domain(DomainId=studio_domain)
print(f'Studio Domain: {studio_domain}; Region: {studio_region}, Studio Type: {studio_type}')

Studio Domain: d-9vucfenphesr; Region: us-west-2, Studio Type: Studio


## Launch the application UI from SageMaker Studio

If you are running this app from SageMaker studio, you need a proxy to run the application. 
Below code will help you lanuch the application after you have started it. Check the value of variable ``studio_type``

In [4]:
space_name = meta_data['SpaceName']
space_settings = sm_client.describe_space(
    DomainId=studio_domain,    
    SpaceName=space_name
)
user_profile_name = space_settings['OwnershipSettings']['OwnerUserProfileName']
studio_url = space_settings['Url']

In [5]:
#If you are running this in a new SageMaker Studio, run the following
proxy_port = 6006
app_url = f'{studio_url}/proxy/{proxy_port}/'
HTML(f"<a href='{app_url}'>Click to launch on New SageMaker Studio</a>")

## Stop & Clean-up
To stop the application, go to terminal and kill the application Ctrl +C
Deactivate conda environment created to exit out

```bash
conda deactivate 
```

You can view all conda environments by running this command
```bash
conda env list
```

Remove the environment created

```bash
conda remove --name aws-notebooklm-app --all
```

To verify that the environment was removed, run
```bash
conda info --envs